<a href="https://colab.research.google.com/github/Maria-Evlakhova/ClinicalRecommendations/blob/main/%D0%9A%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-gigachat gigachain-community unstructured[all-docs] faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 967.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                              chunk_overlap=100)
pdf_file = 'КР359_3.pdf'
loader = UnstructuredPDFLoader(pdf_file)
splitted_data = loader.load_and_split(text_splitter)

In [2]:
len(splitted_data)

483

In [3]:
splitted_data[10]

Document(metadata={'source': 'КР359_3.pdf'}, page_content='План действий при обострении Б А для детей младше 5 лет............................................. 178\n\nВыбор средства доставки лекарственного препарата..........................................................179\n\nАлгоритм перехода на нижнюю ступень лечения у взрослых и подростков с хорошо контролируемой астмой..............................................................................................................181\n\nВторичная профилактика БА (взрослые, подростки, дети).................................................183\n\nПриложение В. Информация для пациента................................................................................186\n\nПриложение Г1-П4. Шкалы оценки, вопросники и другие оценочные инструменты\n\nсостояния пациента, приведенные в клинических рекомендациях.......................................189\n\nПриложение Г1. Тест по контролю над астмой (АСТ)..........................................

In [4]:
splitted_data[11]

Document(metadata={'source': 'КР359_3.pdf'}, page_content='Приложение Г2. Тест по контролю над астмой у детей (с-АСТ)......................................191\n\n4\n\n158\n\nПриложение ГЗ. Опросник по контролю симптомов астмы (АСС2-5)..............................193\n\nПриложение Г4. Опросник по контролю над Б А у детей от 5 лет и младше (ТКАСК) 195\n\nПриложение Г5. Опросник по оценке уровня контроля симптомов бронхиальной астмы у взрослых, подростков и детей от 6 до 11 лет.......................................................................197\n\nПриложение Гб. Дополнительный опросник по оценке уровня контроля симптомов бронхиальной астмы у детей от 6 до 11 лет............................................................................200\n\nПриложение Г7. Дополнительный опросник по оценке уровня контроля симптомов бронхиальной астмы у детей младше 5 лет............................................................................203\n\n5\n\nСписок сокращений\n\nАСТ - АзШта Соп*го1 

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

In [6]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(splitted_data, embedding=embedding)

<timed exec>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 3min 32s, sys: 18 s, total: 3min 50s
Wall time: 4min 13s


In [7]:
q1 = 'Как лечить астму?'
q2 = 'Что включает в себя обследование пациентов?'
q3 = 'Медикаментозная терапия'

In [8]:
vector_store.similarity_search(q1)

[Document(metadata={'source': 'КР359_3.pdf'}, page_content='75\n\nприменения (более 50 лет) и входил в международные и отечественные рекомендации по\n\nлечению астмы при оказании неотложной и стационарной помощи до 2020г:\n\n«внутривенное введение аминофиллина**рекомендуется взрослым и детям старше 2 лет\n\nс тяжелой или угрожающей жизни астмой, не отвечающим на максимальные дозы\n\nингаляционных бронхолитиков и стероидов» [273] Теофиллины (КОЗБА: Ксантины) в\n\nнизких\n\nдозах\n\nпомимо\n\nбронхолитического\n\nдействия\n\nтакже\n\nобладают\n\nпротивовоспалительными свойствами, среди которых наиболее важным является их\n\nспособность восстанавливать гистондеацетилазу-2 (НБАС2), активность которой\n\nприводит к улучшению чувствительности к кортикостероидам ( [274-276]. Ограничение\n\nприменения аминофиллина** имело место в связи с потенциально серьезными побочными\n\nэффектами при быстром (струйном) внутривенном введении и введении в высоких дозах\n\n(более 20 мг/мл) в виде тошноты, рво

In [9]:
vector_store.similarity_search(q2)

[Document(metadata={'source': 'КР359_3.pdf'}, page_content='2.2\n\nФизикальное обследование\n\nРекомендуется всем пациентам, как при подозрении на БА, так н при\n\nустановленном диагнозе проводить физикальное обследование, включающее\n\nоценку общего состояния, визуальное исследование верхних дыхательных\n\nпутей, визуальный осмотр терапевтический и аускультацию терапевтическую\n\nс целью определения\n\nклинико-дннамнческих особенностей заболевания,\n\nвыявления признаков сопутствующей патологии и создания ориентиров в\n\nопределении тяжести состояния [4].\n\nУровень убедительности рекомендаций С (уровень достоверности доказательств - 5)\n\nКомментарии: в случае тяжелого обострения при физикальном осмотре можно\n\nвыявить признаки дыхательной недостаточности: шумное дыхание с дистантными\n\nсвистящими хрипами, вынужденное положение (ортопноэ), тахипноэ, диффузный\n\nцианоз, в акте дыхания участвуют дополнительные группы мышц. Аускультативно\n\nдыхание может быть ослабленным вплоть до «

In [10]:
vector_store.similarity_search(q3)

[Document(metadata={'source': 'КР359_3.pdf'}, page_content='базисной терапии ИГКС. В ограниченных исследованиях было показано, что пациенты,\n\nпродолжавшие биологическую терапию, имели лучшие клинические результаты, чем те,\n\nкто перешел на другую терапию ши прекратил ее, что свидетельствует о важности\n\nправшьного выбора исходного биологического препарата и продолжения терапии\n\n[202,206,220,221].\n\nРекомендуется пациентам с тяжелой БА, неконтролируемой объемом терапин,\n\nсоответствующим\n\n5\n\nступени,\n\nрассмотреть\n\nвопрос\n\nо\n\nназначении\n\nкортикостероидов системного действия (СГКС, код АТХ Н02А) в качестве\n\nдополнения к базисной терапии с целью достижения контроля над\n\nсимптомами заболевания, снижения риска обострений и улучшения функции\n\nлегких [20].\n\nУровень убедительности рекомендаций С (уровень достоверности доказательств - 5)\n\nКомментарии: Назначение СГКС следует рассматривать как терапию\n\n«отчаяния»\n\nисключительную стратегию в случае отсутствия ко

In [23]:
from langchain_community.document_loaders import UnstructuredExcelLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                              chunk_overlap=50)

loader = UnstructuredExcelLoader('mkb10.xlsx')
splitted_xlsx = loader.load_and_split(text_splitter)

In [24]:
len(splitted_xlsx)

3097

In [25]:
splitted_xlsx[5]

Document(metadata={'source': 'mkb10.xlsx'}, page_content='оториноларингологов Да 2025-02-10 17:59:47.347000 Применяется 26_4 Юношеский артрит с системным')

In [26]:
splitted_xlsx1 = loader.load_and_split()

In [27]:
len(splitted_xlsx1)

41

In [28]:
splitted_xlsx1[3]

Document(metadata={'source': 'mkb10.xlsx'}, page_content='– реаниматологов", Межрегиональная ассоциация по клинической микробиологии и антимикробной химиотерапии, Межрегиональная общественная организация «Альянс клинических химиотерапевтов и микробиологов», Межрегиональная общественная организация врачей и медицинских сестер «Сепсис Форум», Общероссийская общественная организация «Российское общество скорой медицинской помощи», Национальная ассоциация специалистов по тромбозам, клинической гемостазиологии и гемореологии, Российская Ассоциация специалистов по лечению хирургических инфекций Да 2024-12-27 15:31:08.733000 Применяется 590_2 Психические и поведенческие расстройства, вызванные употреблением психоактивных веществ Абстинентное состояние (синдром отмены) с делирием F10.4, F11.4, F13.4, F19.4 Взрослые Общественная организация "Российское общество психиатров", Общероссийская общественная организация "Федерация анестезиологов и реаниматологов" Да 2024-12-27 14:56:58.420000 Применяе

In [29]:
vector_store_xlsx = FAISS.from_documents(splitted_xlsx1, embedding=embedding)

In [30]:
q1 = 'Какой код МКБ-10 для дистонии?'
q2 = 'Какой код МКБ-10 для бронхита?'
q3 = 'Какой код МКБ-10 для Ожирение?'

In [32]:
vector_store_xlsx.similarity_search(q1)

[Document(metadata={'source': 'mkb10.xlsx'}, page_content='ID Наименование МКБ-10 Возрастная категория Разработчик Статус одобрения НПС Дата размещения Статус применения 22_3 Нейросенсорная тугоухость у детей H90.3, H90.4, H90.5 Дети Союз педиатров России, Национальная медицинская ассоциация оториноларингологов Да 2025-02-10 17:59:47.347000 Применяется 26_4 Юношеский артрит с системным началом M08.2 Дети Ассоциация детских ревматологов Да 2025-01-30 14:00:05.397000 Применяется 529_2 Острый лимфобластный лейкоз C83.5, C91.0, C95.0 Дети Российское общество детских онкологов и гематологов Да 2025-01-29 17:48:00.327000 Применяется 586_3 Острые миелоидные лейкозы C92.0, C92.3, C92.5, C92.6, C92.7, C92.8, C92.9, C93.0, C94.0, C94.2 Дети Российское общество детских онкологов и гематологов Да 2025-01-28 18:46:09.047000 Применяется 134_2 Хронический лимфоцитарный лейкоз / лимфома из малых лимфоцитов C91.1 Взрослые Ассоциация онкологов России, Российское общество онкогематологов, Ассоциация соде

In [33]:
vector_store_xlsx.similarity_search(q2)

[Document(metadata={'source': 'mkb10.xlsx'}, page_content='Да 2024-12-17 13:57:16.567000 Применяется 603_3 Хроническая обструктивная болезнь легких J44, Z57.2, Z57.3, Z57.4, Z57.5 Взрослые Ассоциация врачей и специалистов медицины труда, Общероссийская общественная организация "Российское научное медицинское общество терапевтов", Российское респираторное общество Да 2024-12-17 00:00:00 Применяется 131_2 Острые миелоидные лейкозы C92.0, C92.3, C92.5, C92.6, C92.7, C92.8, C92.9, C93.0, C94.0, C94.2, C94.7, C95.0 Взрослые Ассоциация содействия развитию гематологии, трансфузиологии и трансплантации костного мозга "Национальное гематологическое общество" Да 2024-12-17 00:00:00 Применяется 43_2 Открытый артериальный проток (ОАП) Q25.0 Взрослые, дети Ассоциация сердечно-сосудистых хирургов России, Общероссийская общественная организация «Российское кардиологическое общество», Всероссийская общественная организация "Ассоциация детских кардиологов России", Общероссийская общественная организаци

In [34]:
vector_store_xlsx.similarity_search(q3)

[Document(metadata={'source': 'mkb10.xlsx'}, page_content='ID Наименование МКБ-10 Возрастная категория Разработчик Статус одобрения НПС Дата размещения Статус применения 22_3 Нейросенсорная тугоухость у детей H90.3, H90.4, H90.5 Дети Союз педиатров России, Национальная медицинская ассоциация оториноларингологов Да 2025-02-10 17:59:47.347000 Применяется 26_4 Юношеский артрит с системным началом M08.2 Дети Ассоциация детских ревматологов Да 2025-01-30 14:00:05.397000 Применяется 529_2 Острый лимфобластный лейкоз C83.5, C91.0, C95.0 Дети Российское общество детских онкологов и гематологов Да 2025-01-29 17:48:00.327000 Применяется 586_3 Острые миелоидные лейкозы C92.0, C92.3, C92.5, C92.6, C92.7, C92.8, C92.9, C93.0, C94.0, C94.2 Дети Российское общество детских онкологов и гематологов Да 2025-01-28 18:46:09.047000 Применяется 134_2 Хронический лимфоцитарный лейкоз / лимфома из малых лимфоцитов C91.1 Взрослые Ассоциация онкологов России, Российское общество онкогематологов, Ассоциация соде

In [35]:
len(splitted_xlsx)

3097

In [36]:
splitted_xlsx[5]

Document(metadata={'source': 'mkb10.xlsx'}, page_content='оториноларингологов Да 2025-02-10 17:59:47.347000 Применяется 26_4 Юношеский артрит с системным')

In [37]:
splitted_xlsx1 = loader.load_and_split()

In [38]:
len(splitted_xlsx1)

41

In [39]:
splitted_xlsx1[3]

Document(metadata={'source': 'mkb10.xlsx'}, page_content='– реаниматологов", Межрегиональная ассоциация по клинической микробиологии и антимикробной химиотерапии, Межрегиональная общественная организация «Альянс клинических химиотерапевтов и микробиологов», Межрегиональная общественная организация врачей и медицинских сестер «Сепсис Форум», Общероссийская общественная организация «Российское общество скорой медицинской помощи», Национальная ассоциация специалистов по тромбозам, клинической гемостазиологии и гемореологии, Российская Ассоциация специалистов по лечению хирургических инфекций Да 2024-12-27 15:31:08.733000 Применяется 590_2 Психические и поведенческие расстройства, вызванные употреблением психоактивных веществ Абстинентное состояние (синдром отмены) с делирием F10.4, F11.4, F13.4, F19.4 Взрослые Общественная организация "Российское общество психиатров", Общероссийская общественная организация "Федерация анестезиологов и реаниматологов" Да 2024-12-27 14:56:58.420000 Применяе

In [40]:
loader = UnstructuredExcelLoader('mkb10.xlsx', mode='elements')
splitted_xlsx2 = loader.load()

In [41]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat

giga = GigaChat(
    # Для авторизации запросов используйте ключ, полученный в проекте GigaChat API
    credentials="",
    verify_ssl_certs=False,
)

messages = [
    SystemMessage(
        content="Ты врач, который помогает пользователю найти ответ на вопрос по клиническим рекомендациям."
    )
]

while(True):
    user_input = input("Пользователь: ")
    if user_input == "пока":
      break
    messages.append(HumanMessage(content=user_input))
    res = giga.invoke(messages)
    messages.append(res)
    print("GigaChat: ", res.content)

Пользователь: код мкб-10 для гастрита
GigaChat:  Гастрит - это воспаление слизистой оболочки желудка, которое может быть острым или хроническим. В МКБ-10 гастрит кодируется следующими рубриками:

K29.0 - Острый геморрагический гастрит;
K29.4 - Хронический поверхностный гастрит;
K29.5 - Хронический атрофический гастрит;
K29.6 - Другие хронические гастриты;
K29.7 - Гастрит неуточненный.
Пользователь: K29.0 - Острый геморрагический гастрит
GigaChat:  Да, вы правы. Код MKC-10 для острого геморрагического гастрита - K29.0.
Пользователь: Как лечить?
GigaChat:  Лечение острого геморрагического гастрита зависит от его причины и тяжести симптомов. Вот общие рекомендации:

1. **Диета**: Избегайте раздражающих продуктов (острые, жирные, жареные блюда) и алкоголя. Ешьте часто небольшими порциями.

2. **Антисекреторные препараты**: Применяются для снижения кислотности желудочного сока. Это могут быть ингибиторы протонной помпы (например, омепразол), блокаторы H2-рецепторов (например, ранитидин).

3

KeyboardInterrupt: Interrupted by user